## Import library

In [38]:
# importing required libraries
import os, sys
import dowhy
from dowhy import CausalModel
import pandas as pd
import numpy as np

import sys
sys.path.insert(1, '/home/dtd/Documents/interpretable_machine_learning/Source Code/my_work/lib')
import numpy as np
import pandas as pd
import logging
import dowhy.datasets
import dowhy
import propensity_score_estimator as pse
import incremental_ps_score_estimator as ipse
import math 
import timeit
import evaluation as ev

from dowhy import CausalModel
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.neighbors import NearestNeighbors
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import explained_variance_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from tqdm import tqdm
from sklearn.metrics import mean_squared_error
from math import sqrt


## Loading Data

In [18]:
DATA_PATH = "/home/dtd/Documents/interpretable_machine_learning/Source Code/data/datasets/IHDP/csv/ihdp_npci_1.csv"

col =  ["treatment", "y_factual", "y_cfactual", "mu0", "mu1" ,]
cov = ["x" + str(i) for i in range(1,26)]
col = col + cov
features = cov + ["treatment"]
data= pd.read_csv(DATA_PATH, header = None, names = col)
data.head()

,treatment,y_factual,y_cfactual,mu0,mu1,x1,x2,x3,x4,x5,...,x16,x17,x18,x19,x20,x21,x22,x23,x24,x25
0,1,5.599916,4.318780,3.268256,6.854457,-0.528603,-0.343455,1.128554,0.161703,-0.316603,...,1,1,1,1,0,0,0,0,0,0
1,0,6.875856,7.856495,6.636059,7.562718,-1.736945,-1.802002,0.383828,2.244320,-0.629189,...,1,1,1,1,0,0,0,0,0,0
2,0,2.996273,6.633952,1.570536,6.121617,-0.807451,-0.202946,-0.360898,-0.879606,0.808706,...,1,0,1,1,0,0,0,0,0,0
3,0,1.366206,5.697239,1.244738,5.889125,0.390083,0.596582,-1.850350,-0.879606,-0.004017,...,1,0,1,1,0,0,0,0,0,0
4,0,1.963538,6.202582,1.685048,6.191994,-1.045229,-0.602710,0.011465,0.161703,0.683672,...,1,1,1,1,0,0,0,0,0,0


In [19]:
data['treatment'] = [bool(x) for x in data['treatment']]

## Model 

In [20]:
model=CausalModel(
        data = data,
        treatment='treatment',
        outcome='y_factual',
        common_causes=cov
        )

INFO:dowhy.causal_graph:If this is observed data (not from a randomized experiment), there might always be missing confounders. Adding a node named "Unobserved Confounders" to reflect this.
INFO:dowhy.causal_model:Model to find the causal effect of treatment ['treatment'] on outcome ['y_factual']


In [21]:
identified_estimand = model.identify_effect()

INFO:dowhy.causal_identifier:Common causes of treatment and outcome:['x21', 'x22', 'x10', 'x8', 'x3', 'x17', 'x5', 'x23', 'x6', 'x12', 'x18', 'x16', 'x24', 'x2', 'x13', 'x15', 'x14', 'x20', 'x7', 'x25', 'x1', 'U', 'x4', 'x19', 'x9', 'x11']


WARN: Do you want to continue by ignoring any unobserved confounders? (use proceed_when_unidentifiable=True to disable this prompt) [y/n] 
Please respond with 'y' or 'n'
WARN: Do you want to continue by ignoring any unobserved confounders? (use proceed_when_unidentifiable=True to disable this prompt) [y/n] 
Please respond with 'y' or 'n'
WARN: Do you want to continue by ignoring any unobserved confounders? (use proceed_when_unidentifiable=True to disable this prompt) [y/n] 
Please respond with 'y' or 'n'
WARN: Do you want to continue by ignoring any unobserved confounders? (use proceed_when_unidentifiable=True to disable this prompt) [y/n] 
Please respond with 'y' or 'n'
WARN: Do you want to continue by ignoring any unobserved confounders? (use proceed_when_unidentifiable=True to disable this prompt) [y/n] 
Please respond with 'y' or 'n'
WARN: Do you want to continue by ignoring any unobserved confounders? (use proceed_when_unidentifiable=True to disable this prompt) [y/n] 
Please resp

INFO:dowhy.causal_identifier:Instrumental variables for treatment and outcome:[]


In [22]:
identified_estimand

## Estimation 

### Linear regression

In [23]:
# Estimate the causal effect and compare it with Average Treatment Effect
estimate = model.estimate_effect(identified_estimand,
        method_name="backdoor.linear_regression", test_significance=True
)

print(estimate)

print("Estimation " + str(estimate.value))
data_1 = data[data["treatment"]==1]
data_0 = data[data["treatment"]==0]
true_effect = np.mean(data_1["y_factual"])- np.mean(data_0["y_factual"])
print("True effect is ", np.mean(data_1["y_factual"])- np.mean(data_0["y_factual"]))

INFO:dowhy.causal_estimator:INFO: Using Linear Regression Estimator
INFO:dowhy.causal_estimator:b: y_factual~treatment+x21+x22+x10+x8+x3+x17+x5+x23+x6+x12+x18+x16+x24+x2+x13+x15+x14+x20+x7+x25+x1+x4+x19+x9+x11


                            OLS Regression Results                            
Dep. Variable:              y_factual   R-squared:                       0.740
Model:                            OLS   Adj. R-squared:                  0.731
Method:                 Least Squares   F-statistic:                     78.91
Date:                Sun, 07 Jun 2020   Prob (F-statistic):          6.10e-191
Time:                        14:35:11   Log-Likelihood:                -1138.1
No. Observations:                 747   AIC:                             2330.
Df Residuals:                     720   BIC:                             2455.
Df Model:                          26                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          2.2206      0.448      4.957      0.0

### Propensity score - K-nearest neighbor

In [24]:
estimate = model.estimate_effect(identified_estimand,
        method_name="backdoor.propensity_score_matching"
)

print("Estimation " + str(estimate.value))
print("True effect is ", np.mean(data_1["y_factual"])- np.mean(data_0["y_factual"]))

INFO:dowhy.causal_estimator:INFO: Using Propensity Score Matching Estimator
INFO:dowhy.causal_estimator:b: y_factual~treatment+x21+x22+x10+x8+x3+x17+x5+x23+x6+x12+x18+x16+x24+x2+x13+x15+x14+x20+x7+x25+x1+x4+x19+x9+x11
/home/dtd/.conda/envs/phd_env/lib/python3.7/site-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Estimation 3.9791388232170393
True effect is  4.021121012430829


In [25]:
abs(true_effect - estimate.value)

## Z-estimator with library

In [30]:
DATA_PATH = "/home/dtd/Documents/interpretable_machine_learning/Source Code/data/datasets/IHDP/csv/ihdp_npci_1.csv"

col =  ["treatment", "y_factual", "y_cfactual", "mu0", "mu1" ,]
cov = ["x" + str(i) for i in range(1,26)]
col = col + cov
features = cov + ["treatment"]
data = pd.read_csv(DATA_PATH, header = None, names = col)
data.head()

,treatment,y_factual,y_cfactual,mu0,mu1,x1,x2,x3,x4,x5,...,x16,x17,x18,x19,x20,x21,x22,x23,x24,x25
0,1,5.599916,4.318780,3.268256,6.854457,-0.528603,-0.343455,1.128554,0.161703,-0.316603,...,1,1,1,1,0,0,0,0,0,0
1,0,6.875856,7.856495,6.636059,7.562718,-1.736945,-1.802002,0.383828,2.244320,-0.629189,...,1,1,1,1,0,0,0,0,0,0
2,0,2.996273,6.633952,1.570536,6.121617,-0.807451,-0.202946,-0.360898,-0.879606,0.808706,...,1,0,1,1,0,0,0,0,0,0
3,0,1.366206,5.697239,1.244738,5.889125,0.390083,0.596582,-1.850350,-0.879606,-0.004017,...,1,0,1,1,0,0,0,0,0,0
4,0,1.963538,6.202582,1.685048,6.191994,-1.045229,-0.602710,0.011465,0.161703,0.683672,...,1,1,1,1,0,0,0,0,0,0


In [34]:
delta_seq = np.linspace(0.5, 8, 50)
treatment = 'treatment'
outcome = 'y_factual'

## Fit treatment 
model_t = LogisticRegression()
model_t.fit(data[cov], data[treatment])

## Fit outcome
model_y = GradientBoostingRegressor(random_state=0, n_estimators = 5000)
model_y.fit(data[features], data[outcome])

GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=0.1, loss='ls', max_depth=3,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=5000,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=0, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

In [35]:
causal_effect = ipse.z_estimator(data, treatment, cov, outcome, delta_seq, model_t, model_y)

100%|██████████| 50/50 [00:01<00:00, 42.88it/s]


In [41]:
data.head()

,treatment,y_factual,y_cfactual,mu0,mu1,x1,x2,x3,x4,x5,...,x23,x24,x25,ps_1,ps_0,predicted_y,treated_cf_outcome,control_cf_outcome,incre_ps_1,incre_ps_0
0,1,5.599916,4.318780,3.268256,6.854457,-0.528603,-0.343455,1.128554,0.161703,-0.316603,...,0,0,0,0.283647,0.716353,5.599095,5.599095,2.221528,0.760058,0.239942
1,0,6.875856,7.856495,6.636059,7.562718,-1.736945,-1.802002,0.383828,2.244320,-0.629189,...,0,0,0,0.067996,0.932004,6.869445,8.350381,6.869445,0.368549,0.631451
2,0,2.996273,6.633952,1.570536,6.121617,-0.807451,-0.202946,-0.360898,-0.879606,0.808706,...,0,0,0,0.107668,0.892332,2.990300,7.359767,2.990300,0.491165,0.508835
3,0,1.366206,5.697239,1.244738,5.889125,0.390083,0.596582,-1.850350,-0.879606,-0.004017,...,0,0,0,0.129877,0.870123,1.366908,4.000166,1.366908,0.544234,0.455766
4,0,1.963538,6.202582,1.685048,6.191994,-1.045229,-0.602710,0.011465,0.161703,0.683672,...,0,0,0,0.092954,0.907046,1.984696,6.987097,1.984696,0.450501,0.549499


In [36]:
print("Estimation " + str(causal_effect))
print("True effect is ", np.mean(data_1["y_factual"])- np.mean(data_0["y_factual"]))
print("MAE {}".format(abs(true_effect - causal_effect)))

pehe = ev.PEHE(data, data['y_factual'], data['y_cfactual'])
print("PEHE {}".format()

Estimation 4.024316717821617
True effect is  4.021121012430829
MAE 0.0031957053907882838


In [42]:
data.head(100)

,treatment,y_factual,y_cfactual,mu0,mu1,x1,x2,x3,x4,x5,...,x23,x24,x25,ps_1,ps_0,predicted_y,treated_cf_outcome,control_cf_outcome,incre_ps_1,incre_ps_0
0,1,5.599916,4.318780,3.268256,6.854457,-0.528603,-0.343455,1.128554,0.161703,-0.316603,...,0,0,0,0.283647,0.716353,5.599095,5.599095,2.221528,0.760058,0.239942
1,0,6.875856,7.856495,6.636059,7.562718,-1.736945,-1.802002,0.383828,2.244320,-0.629189,...,0,0,0,0.067996,0.932004,6.869445,8.350381,6.869445,0.368549,0.631451
2,0,2.996273,6.633952,1.570536,6.121617,-0.807451,-0.202946,-0.360898,-0.879606,0.808706,...,0,0,0,0.107668,0.892332,2.990300,7.359767,2.990300,0.491165,0.508835
3,0,1.366206,5.697239,1.244738,5.889125,0.390083,0.596582,-1.850350,-0.879606,-0.004017,...,0,0,0,0.129877,0.870123,1.366908,4.000166,1.366908,0.544234,0.455766
4,0,1.963538,6.202582,1.685048,6.191994,-1.045229,-0.602710,0.011465,0.161703,0.683672,...,0,0,0,0.092954,0.907046,1.984696,6.987097,1.984696,0.450501,0.549499
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0,4.109239,8.285439,4.242322,7.115311,0.426830,0.996346,-0.733261,-0.879606,0.371086,...,0,0,0,0.289570,0.710430,4.111574,7.511011,4.111574,0.765301,0.234699
96,0,1.329745,6.518780,1.475950,6.059502,0.856992,0.596582,-1.105624,0.161703,1.371361,...,0,0,0,0.131962,0.868038,1.337994,5.052212,1.337994,0.548773,0.451227
97,0,1.638059,6.369190,2.878175,6.727356,0.424669,0.596582,-0.733261,1.203011,1.433878,...,0,0,0,0.175305,0.824695,1.635767,5.780589,1.635767,0.629705,0.370295
98,0,1.522918,5.022604,1.127707,5.790386,0.403053,0.196818,0.011465,-0.879606,1.746464,...,0,0,0,0.219242,0.780758,1.527590,6.127063,1.527590,0.691971,0.308029


## Reference
https://microsoft.github.io/dowhy/dowhy_ihdp_data_example.html